In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np

In [ ]:
window = tk.Tk()
window.title("Face Recognition System")
window.config(background='skyblue')

l1 = tk.Label(window,text="Name",font=("Algerian",20),bg='sky blue')
l1.grid(column=0, row=0)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window,text="Age",font=("Algerian",20),bg='sky blue')
l2.grid(column=0, row=1)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1, row=1)

#TRAIN THE CLASSIFIER
def train_classifier():
    data_dir="C:/Users/Lenovo/Downloads/faace Recognition/data"
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    #train the classifier 
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml") 
    messagebox.shoeinfo('Result','Training dataset completed')
b1=tk.Button(window,text="Training",font=("algerian",20),bg='light blue',fg='black',command= train_classifier)
b1.grid(column=0,row=4)

#DETECT THE FACE
def detect_face():
    def draw_boundary(img,classifier,scaleFactor,minNeighbors,color,text,clf):
        gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image,scaleFactor,minNeighbors)
        names = ['Dhairya','Kirti']
        coords = []

        for(x,y,w,h) in features:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            id,confidence = clf.predict(gray_image[y:y+h,x:x+w])
            
            if (confidence < 100):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))
        
           
            cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
            cv2.putText(
                    img, 
                    str(confidence), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   )  
            

          #  if confidence>60:
               # if id==1:
                    #cv2.putText(img,"Dhairya",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
               # if id==2:
                    #cv2.putText(img,"Kirti",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
           ## else:
               # cv2.putText(img,"UNKNOWN",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),1,cv2.LINE_AA)

            #coords=[x,y,w,h]
        #return coords """
 
    def recognize(img,clf,faceCascade):
        coords = draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
        return img
    faceCascade =  cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf= cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)

    while True:
        ret,img = video_capture.read()
        img = recognize(img,clf,faceCascade)
        cv2.imshow("face detection",img)

        if cv2.waitKey(1)==13:
            break

    video_capture.release()
    cv2.destroyAllWindows()
b2=tk.Button(window,text="Detect",font=("algerian",20),bg='light blue',fg='black',command=detect_face)
b2.grid(column=1,row=4)

#GENERATING DATASET 
def generate_dataset():
    if(t1.get()=="" or t2.get()==""):
        messagebox.showinfo('Result','Please Provide Complete Details')
    else:
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray,1.3,5)
            #scaling factor = 1.3
            #Minimum Neighbor = 5
            
            if faces is ():
                return None
            for(x,y,w,h) in faces:
                cropped_face = img[y:y+h,x:x+w]
                return cropped_face
    
    cap = cv2.VideoCapture(0)
    id = 1
    img_id = 0
    
    while True:
        ret,frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame),(200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_path,face)
            cv2.putText(face, str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1 ,(0,255,0),2)
            #(50,50) IS THE ORIGIN POINT FROM WHERE TEXT IS TO BE WRITTEN
            #FONT SCALE =1
            #THICKNESS=2
            cv2.imshow("Cropped Face", face)
            if cv2.waitKey(1)==13 or int(img_id)==200:
                break
    cap.release()
    cv2.desstroyAllWindows()
    messagebox.showinfo('Results','Generating dataset completed!')
    

b3=tk.Button(window,text="Generate Dataset",font=("algerian",20),bg='light blue',fg='black',command=generate_dataset)
b3.grid(column=2,row=4)

window.geometry("900x400")
window.mainloop()